In [1]:
# Importing dependencies
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from art.utils import load_dataset, to_categorical , preprocess
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


In [2]:
# Loading the dataset
(train_images_original, train_labels_original), (test_images_original, test_labels_original), min, max = load_dataset(name="mnist")


In [3]:
# Function for creating model
# For the next of the illustrations we will use CNN model

def create_model():
    # Defining the model
    model = tf.keras.models.Sequential([
        Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(28, 28, 1)),
        Conv2D(filters=32, kernel_size=3, activation="relu", strides=2),
        Conv2D(filters=64, kernel_size=3, activation="relu"),
        Conv2D(filters=64, kernel_size=3, activation="relu", strides=2),
        Flatten(),
        Dense(units=100, activation="relu"),
        Dense(units=10, activation="softmax")
    ])

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model


In [4]:

# Setting aside a subset of the source dataset for the original model
original_images = train_images_original[:50000]
original_images_labels = train_labels_original[:50000]

# Using the rest of the source dataset for the stolen model
train_images_stolen = train_images_original[50000:]
train_labels_stolen = train_labels_original[50000:]

In [5]:
# Training the original model on its training subset
model_original = create_model()
model_original.fit(
    x=original_images,
    y=original_images_labels,
    epochs=10,
    batch_size=256
)





Train on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 8s 160us/sample - loss: 0.3387 - accuracy: 0.8952
Epoch 2/10
50000/50000 [==============================] - 7s 146us/sample - loss: 0.0784 - accuracy: 0.9760
Epoch 3/10
50000/50000 [==============================] - 7s 146us/sample - loss: 0.0526 - accuracy: 0.9835
Epoch 4/10
50000/50000 [==============================] - 7s 144us/sample - loss: 0.0394 - accuracy: 0.9877
Epoch 5/10
50000/50000 [==============================] - 7s 149us/sample - loss: 0.0301 - accuracy: 0.9906
Epoch 6/10
50000/50000 [==============================] - 7s 146us/sample - loss: 0.0245 - accuracy: 0.9921
Epoch 7/10
50000/50000 [==============================] - 7s 148us/sample - loss: 0.0200 - accuracy: 0.9939
Epoch 8/10
50000/50000 [==============================] - 8s 151us/sample - loss: 0.0152 - accuracy: 0.9953
Epoch 9/10
50000/50000 [==============================] - 7s 148us/sample - loss: 0.0132 - accuracy: 0.9957
E

In [6]:
from art.estimators.classification import KerasClassifier

# Wrapping the model in the ART KerasClassifier class
classifier_original = KerasClassifier(
    model=model_original,
    clip_values=(min, max))

In [7]:
# Creating a reference model for theft
model_stolen = KerasClassifier(
    model=create_model(), 
    clip_values=(min, max)
    )

In [8]:
from art.attacks.extraction import CopycatCNN
# Creating the "neural net thief" object
# that will steal the original classifier
copycat = CopycatCNN(
    batch_size_fit=256,
    batch_size_query=256,
    nb_epochs=20,
    nb_stolen=len(train_images_stolen),
    classifier=classifier_original
    )


In [9]:
# Extracting a thieved classifier
# by training the reference model
stolen_classifier = copycat.extract(
    x=train_images_stolen, 
    y=train_labels_stolen, 
    thieved_classifier=model_stolen
    )

C:\Users\Darshan Mourya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [10]:
# Testing the performance of the original classifier
score_original = classifier_original._model.evaluate(
    x=test_images_original, 
    y=test_labels_original
    )

# Testing the performance of the stolen classifier
score_stolen = stolen_classifier._model.evaluate(
    x=test_images_original, 
    y=test_labels_original
    )

# Comparing test losses
print(f"Original test loss: {score_original[0]:.2f} " 
      f"vs stolen test loss: {score_stolen[0]:.2f}")

# Comparing test accuracies
print(f"Original test accuracy: {score_original[1]:.2f} " 
      f"vs stolen test accuracy: {score_stolen[1]:.2f}")

C:\Users\Darshan Mourya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Original test loss: 0.05 vs stolen test loss: 0.12
Original test accuracy: 0.99 vs stolen test accuracy: 0.98


just by using 50% of output for the train images we are able to reach 98% accuarcy with the model